In [1]:
using StanModels

ProjDir = rel_path_s("..", "scripts", "12")
cd(ProjDir)

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col log_pop, set log() for population data

In [2]:
d[:log_pop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;
first(d[[:culture, :population, :log_pop, :society]], 5)

m12_6 = "
data {
    int N;
    int N_societies;
    int total_tools[N];
    real logpop[N];
    int society[N];
}
parameters{
    real a;
    real bp;
    vector[N_societies] a_society;
    real<lower=0> sigma_society;
}
model{
    vector[N_societies] mu;
    sigma_society ~ cauchy( 0 , 1 );
    a_society ~ normal( 0 , sigma_society );
    bp ~ normal( 0 , 1 );
    a ~ normal( 0 , 10 );
    for ( i in 1:N ) {
        mu[i] = a + a_society[society[i]] + bp * logpop[i];
        mu[i] = exp(mu[i]);
    }
    total_tools ~ poisson( mu );
}
";

Define the Stanmodel and set the output format to :mcmcchains.

In [3]:
stanmodel = Stanmodel(name="m12.6",  model=m12_6, output_format=:mcmcchains);


File /Users/rob/.julia/dev/StanModels/scripts/12/tmp/m12.6.stan will be updated.



Input data for cmdstan

In [4]:
m12_6_data = Dict("N" => size(d, 1),"N_societies" => 10,
"total_tools" => d[:total_tools], "logpop" => d[:log_pop],
"society" => d[:society]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_data, ProjDir, diagnostics=false, summary=false, CmdStanDir=CMDSTAN_HOME);

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in '/Users/rob/.julia/dev/StanModels/scripts/12/tmp/m12.6.stan' at line 17)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.



Describe the draws

In [6]:
describe(chn)

2-element Array{ChainDataFrame,1}

Summary Statistics
. Omitted printing of 1 columns
│ Row │ parameters    │ mean       │ std       │ naive_se   │ mcse       │
│     │ Symbol        │ Float64    │ Float64   │ Float64    │ Float64    │
├─────┼───────────────┼────────────┼───────────┼────────────┼────────────┤
│ 1   │ a             │ 1.09736    │ 0.73914   │ 0.0116868  │ 0.0221613  │
│ 2   │ a_society.1   │ -0.197777  │ 0.237919  │ 0.00376182 │ 0.00739322 │
│ 3   │ a_society.2   │ 0.0456694  │ 0.21688   │ 0.00342918 │ 0.00449798 │
│ 4   │ a_society.3   │ -0.0502063 │ 0.194344  │ 0.00307285 │ 0.00451446 │
│ 5   │ a_society.4   │ 0.319274   │ 0.199118  │ 0.00314834 │ 0.00680113 │
│ 6   │ a_society.5   │ 0.0432777  │ 0.178054  │ 0.00281528 │ 0.00412792 │
│ 7   │ a_society.6   │ -0.311483  │ 0.208608  │ 0.00329838 │ 0.00774609 │
│ 8   │ a_society.7   │ 0.143757   │ 0.181245  │ 0.00286573 │ 0.00456763 │
│ 9   │ a_society.8   │ -0.170802  │ 0.189669  │ 0.00299893 │ 0.00455143 │
│ 10  │ a_soci

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*